In [1]:
from PIL import Image
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [2]:
# get images from pf_ext
labels = os.path.join('Pf_ext_01.json')

count = 1
phase_count = {
    "train": 1,
    "test": 1
}
phase_count["train"] = 1
phase_count["test"] = 1

phase = "train"
# degs = [45,90,135,180,225,270,315]
degs = [90,180,270,360,450,540,630]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "2":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "1":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1
            for deg in degs:
                name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (((x2-x1)/2)+x1)/aug_w
                    bb_yc = (((y2-y1)/2)+y1)/aug_h
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                    f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015250.jpg
pf-train-0000000001
2 90
3 180
4 270
5 360
6 450
7 540
8 630
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015251.jpg
pf-train-0000000009
10 90
11 180
12 270
13 360
14 450
15 540
16 630
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015252.jpg
pf-train-0000000017
18 90
19 180
20 270
21 360
22 450
23 540
24 630
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015253.jpg
pf-train-0000000025
26 90
27 180
28 270
29 360
30 450
31 540
32 630
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015254.jpg
pf-train-0000000033
34 90
35 180
36 270
37 360
38 450
39 540
40 630
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015255.jpg
pf-train-0000000041
42 90
43 180
44 270
45 360
46 450
47 630
7 https://raw.githubusercontent.com/RTANC/malaria_dataset/ma

396 360
397 450
398 540
399 630
51 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015300.jpg
pf-train-0000000400
401 90
402 180
403 270
404 360
405 450
406 540
407 630
52 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015301.jpg
pf-train-0000000408
409 90
410 180
411 270
412 360
413 450
414 540
415 630
53 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015302.jpg
pf-train-0000000416
417 90
418 180
419 270
420 360
421 450
422 540
423 630
54 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015303.jpg
pf-train-0000000424
425 90
426 180
427 270
428 360
429 450
430 540
431 630
55 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015304.jpg
pf-train-0000000432
433 90
434 180
435 270
436 360
437 450
438 540
439 630
56 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015305.jpg
pf-train-0000000440
441 90
442 180
443 27

791 630
100 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015349.jpg
pf-train-0000000792
793 90
794 180
795 270
796 360
797 450
798 540
799 630
101 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015350.jpg
pf-train-0000000800
801 90
802 180
803 270
804 360
805 450
806 540
807 630
102 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015351.jpg
pf-train-0000000808
809 90
810 180
811 270
812 360
813 450
814 540
815 630
103 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015352.jpg
pf-train-0000000816
817 90
818 180
819 270
820 360
821 450
822 540
823 630
104 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015353.jpg
pf-train-0000000824
825 90
826 180
827 270
828 360
829 450
830 540
831 630
105 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015354.jpg
pf-train-0000000832
833 90
834 180
835 270
836 360
837 450


In [3]:
print(phase_count["train"])

118
936


In [4]:
# get image from pf_01
labels = os.path.join('pf_01.json')
count = 1
# degs = [45,90,135,180,225,270,315]
with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1
            for deg in degs:
                name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                    f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(deg,name))
                    phase_count[phase] += 1

1 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j17.jpg
90 pf-train-0000000937
180 pf-train-0000000938
450 pf-train-0000000939
630 pf-train-0000000940
2 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j21.jpg
90 pf-train-0000000942
360 pf-train-0000000943
450 pf-train-0000000944
3 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j22.jpg
4 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j15.jpg
90 pf-train-0000000947
180 pf-train-0000000948
270 pf-train-0000000949
360 pf-train-0000000950
450 pf-train-0000000951
540 pf-train-0000000952
630 pf-train-0000000953
5 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j16.jpg
90 pf-train-0000000955
180 pf-train-0000000956
270 pf-train-0000000957
360 pf-train-0000000958
450 pf-train-000000095

360 pf-train-0000001150
450 pf-train-0000001151
540 pf-train-0000001152
630 pf-train-0000001153
37 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j20.jpg
90 pf-train-0000001155
180 pf-train-0000001156
270 pf-train-0000001157
360 pf-train-0000001158
450 pf-train-0000001159
540 pf-train-0000001160
630 pf-train-0000001161
38 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j21.jpg
180 pf-train-0000001163
270 pf-train-0000001164
360 pf-train-0000001165
450 pf-train-0000001166
540 pf-train-0000001167
630 pf-train-0000001168
39 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j22.jpg
90 pf-train-0000001170
180 pf-train-0000001171
270 pf-train-0000001172
360 pf-train-0000001173
450 pf-train-0000001174
540 pf-train-0000001175
630 pf-train-0000001176
40 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Ex

630 pf-train-0000001384
67 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j50.jpg
90 pf-train-0000001386
180 pf-train-0000001387
270 pf-train-0000001388
360 pf-train-0000001389
450 pf-train-0000001390
540 pf-train-0000001391
630 pf-train-0000001392
68 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j51.jpg
90 pf-train-0000001394
180 pf-train-0000001395
270 pf-train-0000001396
360 pf-train-0000001397
450 pf-train-0000001398
540 pf-train-0000001399
630 pf-train-0000001400
69 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j52.jpg
90 pf-train-0000001402
180 pf-train-0000001403
270 pf-train-0000001404
360 pf-train-0000001405
450 pf-train-0000001406
540 pf-train-0000001407
630 pf-train-0000001408
70 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j53.jpg
71 http://map.ku.ac.th/imglabel

450 pf-train-0000001606
630 pf-train-0000001607
101 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j33.jpg
90 pf-train-0000001609
180 pf-train-0000001610
270 pf-train-0000001611
360 pf-train-0000001612
450 pf-train-0000001613
540 pf-train-0000001614
630 pf-train-0000001615
102 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j34.jpg
90 pf-train-0000001617
180 pf-train-0000001618
270 pf-train-0000001619
360 pf-train-0000001620
450 pf-train-0000001621
540 pf-train-0000001622
630 pf-train-0000001623
103 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j35.jpg
90 pf-train-0000001625
180 pf-train-0000001626
270 pf-train-0000001627
360 pf-train-0000001628
450 pf-train-0000001629
540 pf-train-0000001630
630 pf-train-0000001631
104 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j36.jpg
90 

630 pf-train-0000001841
131 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j63.jpg
90 pf-train-0000001843
180 pf-train-0000001844
270 pf-train-0000001845
360 pf-train-0000001846
450 pf-train-0000001847
540 pf-train-0000001848
630 pf-train-0000001849
132 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j64.jpg
90 pf-train-0000001851
180 pf-train-0000001852
270 pf-train-0000001853
360 pf-train-0000001854
450 pf-train-0000001855
540 pf-train-0000001856
630 pf-train-0000001857
133 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j65.jpg
90 pf-train-0000001859
180 pf-train-0000001860
270 pf-train-0000001861
360 pf-train-0000001862
450 pf-train-0000001863
540 pf-train-0000001864
630 pf-train-0000001865
134 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j66.jpg
270 pf-train-0000001867
360

...

90 pf-train-0000005088
180 pf-train-0000005089
270 pf-train-0000005090
360 pf-train-0000005091
450 pf-train-0000005092
540 pf-train-0000005093
630 pf-train-0000005094
562 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j40.jpg
90 pf-train-0000005096
180 pf-train-0000005097
270 pf-train-0000005098
360 pf-train-0000005099
450 pf-train-0000005100
540 pf-train-0000005101
630 pf-train-0000005102
563 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j41.jpg
90 pf-train-0000005104
180 pf-train-0000005105
270 pf-train-0000005106
360 pf-train-0000005107
450 pf-train-0000005108
540 pf-train-0000005109
630 pf-train-0000005110
564 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j43.jpg
90 pf-train-0000005112
180 pf-train-0000005113
270 pf-train-0000005114
360 pf-train-0000005115
450 pf-train-0000005116
540 pf-train-0000005117
630 pf-train-0000005

90 pf-train-0000005321
180 pf-train-0000005322
270 pf-train-0000005323
360 pf-train-0000005324
450 pf-train-0000005325
540 pf-train-0000005326
630 pf-train-0000005327
592 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j73.jpg
90 pf-train-0000005329
180 pf-train-0000005330
270 pf-train-0000005331
360 pf-train-0000005332
450 pf-train-0000005333
540 pf-train-0000005334
630 pf-train-0000005335
593 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j16.jpg
90 pf-train-0000005337
180 pf-train-0000005338
270 pf-train-0000005339
360 pf-train-0000005340
450 pf-train-0000005341
540 pf-train-0000005342
630 pf-train-0000005343
594 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j18.jpg
90 pf-train-0000005345
180 pf-train-0000005346
270 pf-train-0000005347
360 pf-train-0000005348
450 pf-train-0000005349
540 pf-train-0000005350
630 pf-train-0000005

90 pf-train-0000005555
180 pf-train-0000005556
270 pf-train-0000005557
360 pf-train-0000005558
450 pf-train-0000005559
540 pf-train-0000005560
630 pf-train-0000005561
622 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j48.jpg
90 pf-train-0000005563
180 pf-train-0000005564
270 pf-train-0000005565
360 pf-train-0000005566
450 pf-train-0000005567
540 pf-train-0000005568
630 pf-train-0000005569
623 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j49.jpg
90 pf-train-0000005571
180 pf-train-0000005572
270 pf-train-0000005573
360 pf-train-0000005574
450 pf-train-0000005575
540 pf-train-0000005576
630 pf-train-0000005577
624 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j50.jpg
90 pf-train-0000005579
180 pf-train-0000005580
270 pf-train-0000005581
360 pf-train-0000005582
450 pf-train-0000005583
540 pf-train-0000005584
630 pf-train-0000005

90 pf-train-0000005792
180 pf-train-0000005793
270 pf-train-0000005794
360 pf-train-0000005795
450 pf-train-0000005796
540 pf-train-0000005797
630 pf-train-0000005798
652 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j18.jpg
90 pf-train-0000005800
180 pf-train-0000005801
270 pf-train-0000005802
360 pf-train-0000005803
450 pf-train-0000005804
540 pf-train-0000005805
630 pf-train-0000005806
653 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j19.jpg
90 pf-train-0000005808
180 pf-train-0000005809
270 pf-train-0000005810
360 pf-train-0000005811
450 pf-train-0000005812
540 pf-train-0000005813
630 pf-train-0000005814
654 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j21.jpg
90 pf-train-0000005816
180 pf-train-0000005817
270 pf-train-0000005818
360 pf-train-0000005819
450 pf-train-0000005820
540 pf-train-0000005821
630 pf-train-0000005

540 pf-train-0000006025
630 pf-train-0000006026
682 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j51.jpg
90 pf-train-0000006028
180 pf-train-0000006029
270 pf-train-0000006030
360 pf-train-0000006031
450 pf-train-0000006032
540 pf-train-0000006033
630 pf-train-0000006034
683 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j52.jpg
90 pf-train-0000006036
180 pf-train-0000006037
270 pf-train-0000006038
360 pf-train-0000006039
450 pf-train-0000006040
540 pf-train-0000006041
630 pf-train-0000006042
684 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j53.jpg
90 pf-train-0000006044
180 pf-train-0000006045
270 pf-train-0000006046
360 pf-train-0000006047
450 pf-train-0000006048
540 pf-train-0000006049
630 pf-train-0000006050
685 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j55.jpg
90 

In [5]:
print(phase_count["train"])

6147


In [ ]:
# get image from pf_test
labels = os.path.join('pf_test.json')

count = 1
phase_count = {
    "test": 1
}

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

In [ ]:
# get image from pf_02 (Test set)
labels = os.path.join('pf_02.json')

count = 1
phase_count = {
    "test": 11
}

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0 and count <= 100:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
#             rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "2":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#                     rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            phase_count[phase] += 1
#             img = np.array(img)
#             bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1

In [ ]:
# summary training set
# files = glob.glob("images/pf-train-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
#         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [ ]:
# clear empty box
# files = glob.glob("images/*")
# count = 0
# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     txt = list(f)
#     f.close()
#     if len(txt) == 0:
#         print(img)
#         os.remove("labels/"+name+".txt")
#         os.remove(file)
#         count += 1
# print(count)

In [ ]:
# labels = os.path.join('Pf','pf_01.json')
# count = 0
# with open(labels,'r') as json_file:
#     data = json.load(json_file)
#     images = data["_via_img_metadata"]
#     for key in images:
#         url = images[key]["filename"]
#         if count >= 120 and count <= 240:
#             name = "{:010d}".format(count)
#             print("{} {}".format(count,url))
#             response = requests.get(url)
#             img = Image.open(BytesIO(response.content))
#             w, h = img.size
#             img.save(os.path.join('data','samples',name+".jpg"))
#         count += 1
    

In [ ]:
# count = 9784
# for i in range(count,12583):
#     name = "{:010d}".format(i)
#     print(name)
#     os.remove(os.path.join("data/custom/images/",name+".jpg"))
#     os.remove(os.path.join("data/custom/labels/",name+".txt"))